### Test case LVV-T26: Verify implementation of Maximum Likelihood Values and Covariances

Verify that quantities delivered by all measurement algorithms include maximum likelihood values and covariances.

**Discussion**: We will verify this requirement using the DP1 Qserv database, accessible via the TAP query service. 

In [12]:
import matplotlib.pyplot as plt
import numpy as np

from lsst.rsp import get_tap_service

Instantiate the TAP service.

In [2]:
service = get_tap_service('tap')

#### Retrieve the schema for the entire Object table:

In [5]:
query = "SELECT column_name, datatype, description, unit " \
        "FROM tap_schema.columns " \
        "WHERE table_name = 'dp1.Object'"
columns = service.search(query).to_table()

In [6]:
columns

column_name,datatype,description,unit
object,object,object,object
coord_dec,double,Fiducial ICRS Declination of centroid used for database indexing,deg
coord_decErr,float,Error in fiducial ICRS Declination of centroid,deg
coord_ra,double,Fiducial ICRS Right Ascension of centroid used for database indexing,deg
coord_ra_dec_Cov,float,Covariance between fiducial ICRS Right Ascension and Declination of centroid,deg**2
coord_raErr,float,Error in fiducial ICRS Right Ascension of centroid,deg
deblend_failed,boolean,Deblender failed to deblend this source,
deblend_incompleteData,boolean,One or more bands were not deblended due to an inability to model the PSF.,
deblend_isolatedParent,boolean,Deblender skipped this footprint because there was only a single peak,
deblend_iterations,int,Number of iterations during deblending,


#### Select boolean flag columns

In [17]:
bool_flag = np.array(['boolean' in datatype for datatype in columns['datatype']])
columns[bool_flag]

column_name,datatype,description,unit
object,object,object,object
deblend_failed,boolean,Deblender failed to deblend this source,
deblend_incompleteData,boolean,One or more bands were not deblended due to an inability to model the PSF.,
deblend_isolatedParent,boolean,Deblender skipped this footprint because there was only a single peak,
deblend_masked,boolean,Deblender skipped this source because there were too many masked pixels.,
deblend_parentTooBig,boolean,Deblender skipped this source because the parent footprint was too large.,
deblend_skipped,boolean,Deblender skipped this source,
deblend_tooManyPeaks,boolean,Deblender skipped this source because there were too many peaks in the Footprint.,
detect_fromBlend,boolean,This source is deblended from a parent with more than one child.,
detect_isDeblendedModelSource,boolean,True if source has no children and is in the inner region of a coadd patch and is in the inner region of a coadd tract and is not detected in a pseudo-filter (see config.pseudoFilterList) and is a deblended child,


We see that there are many flags, including flags related to the measurement of `psfFlux` (e.g., `z_psfFlux_flag`, `z_psfFlux_flag_apCorr`, `z_psfFLux_flag_edge`, and `z_psfFlux_flag_noGoodPixels`). Additionally, there are flags related to position and shape measurement, detection, deblending, and pixel values.

#### Select coordinate columns (i.e., those with "ra" or "dec" in their names):

RA, Dec coordinates are examples of two measurements that are jointly measured, and should thus have covariances. Confirm that the covariances are reported.

In [41]:
coo_flag = np.array(['coord' in colname for colname in columns['column_name']])
coord_cols = (coo_flag)
columns[coord_cols]

column_name,datatype,description,unit
object,object,object,object
coord_dec,double,Fiducial ICRS Declination of centroid used for database indexing,deg
coord_decErr,float,Error in fiducial ICRS Declination of centroid,deg
coord_ra,double,Fiducial ICRS Right Ascension of centroid used for database indexing,deg
coord_ra_dec_Cov,float,Covariance between fiducial ICRS Right Ascension and Declination of centroid,deg**2
coord_raErr,float,Error in fiducial ICRS Right Ascension of centroid,deg


We see that the covariance is reported in the column `coord_ra_dec_Cov`.

#### Select flux columns:

We will use the flux columns to demonstrate that measured quantities report associated errors in addition to their maximum likelihood measurement.

In [43]:
# flux_flag = np.array([('Flux' in colname)&('area' not in colname) for colname in columns['column_name']])
flux_flag = np.array([str.endswith(colname, 'Flux') for colname in columns['column_name']])
columns[flux_flag & ~bool_flag]

column_name,datatype,description,unit
object,object,object,object
g_ap03Flux,float,Flux within 3.0-pixel aperture. Forced on g-band.,nJy
g_ap06Flux,float,Flux within 6.0-pixel aperture. Forced on g-band.,nJy
g_ap09Flux,float,Flux within 9.0-pixel aperture. Forced on g-band.,nJy
g_ap12Flux,float,Flux within 12.0-pixel aperture. Forced on g-band.,nJy
g_ap17Flux,float,Flux within 17.0-pixel aperture. Forced on g-band.,nJy
g_ap25Flux,float,Flux within 25.0-pixel aperture. Forced on g-band.,nJy
g_ap35Flux,float,Flux within 35.0-pixel aperture. Forced on g-band.,nJy
g_ap50Flux,float,Flux within 50.0-pixel aperture. Forced on g-band.,nJy
g_ap70Flux,float,Flux within 70.0-pixel aperture. Forced on g-band.,nJy


#### Confirm that all flux columns have an associated error column:

In [44]:
fluxcolnames = []
fluxerrcolnames = []

for col in columns[flux_flag & ~bool_flag]['column_name']:
    fluxcolnames.append(col)
    if (col+'Err') in columns['column_name']:
        fluxerrcolnames.append(col+'Err')
    else:
        fluxerrcolnames.append('None')

nfluxcols_without_err = np.sum(np.array(fluxerrcolnames) == 'None')
if nfluxcols_without_err == 0:
    print('All flux columns have associated errors.')
else:
    print(f"{nfluxcols_without_err} flux columns do not have associated errors.")


All flux columns have associated errors.


#### Select model-fit columns

The `multiProFit` model-fitting code outputs "twoGaussian" fits and multiband "sersic" fits. Select these and examine them to confirm that model chi-squared (goodness of fit) values are reported.

In [52]:
twogauss_flag = np.array(['TwoGaussian' in colname for colname in columns['column_name']])
sersic_flag = np.array(['ersic' in colname for colname in columns['column_name']])
chisq_flag = np.array(['chisq' in colname for colname in columns['column_name']])
modelfit_cols = (twogauss_flag | sersic_flag)
print('All modelfit columns:\n', columns[modelfit_cols]['column_name'].value)
print('\n\n')
chisq_modelfit_cols = (twogauss_flag | sersic_flag) & chisq_flag
print('Modelfit chi-squared columns:\n', columns[chisq_modelfit_cols]['column_name'].value)

All modelfit columns:
 ['g_psfModel_TwoGaussian_chisq_reduced'
 'g_psfModel_TwoGaussian_gauss1_fluxfrac'
 'g_psfModel_TwoGaussian_gauss1_rho'
 'g_psfModel_TwoGaussian_gauss1_sigma_x'
 'g_psfModel_TwoGaussian_gauss1_sigma_y'
 'g_psfModel_TwoGaussian_gauss2_rho'
 'g_psfModel_TwoGaussian_gauss2_sigma_x'
 'g_psfModel_TwoGaussian_gauss2_sigma_y' 'g_psfModel_TwoGaussian_n_iter'
 'g_psfModel_TwoGaussian_no_inputs_flag'
 'g_psfModel_TwoGaussian_unknown_flag' 'g_sersicFlux' 'g_sersicFluxErr'
 'i_psfModel_TwoGaussian_chisq_reduced'
 'i_psfModel_TwoGaussian_gauss1_fluxfrac'
 'i_psfModel_TwoGaussian_gauss1_rho'
 'i_psfModel_TwoGaussian_gauss1_sigma_x'
 'i_psfModel_TwoGaussian_gauss1_sigma_y'
 'i_psfModel_TwoGaussian_gauss2_rho'
 'i_psfModel_TwoGaussian_gauss2_sigma_x'
 'i_psfModel_TwoGaussian_gauss2_sigma_y' 'i_psfModel_TwoGaussian_n_iter'
 'i_psfModel_TwoGaussian_no_inputs_flag'
 'i_psfModel_TwoGaussian_unknown_flag' 'i_sersicFlux' 'i_sersicFluxErr'
 'r_psfModel_TwoGaussian_chisq_reduced'
 'r_psf

We see that each of the model-fitting outputs has an associated chi-squared column.

### Results:

We have demonstrated that the DP1 `Object` table contains maximum likelihood values and their associated errors for all measured quantities. We have also shown that jointly-measured quantities report covariances, and that model-fit measurements report goodness of fit.

While we have only demonstrated this for the `Object` table, we take it as representative of measurement algorithm outputs for all methods in the Science Pipelines. With this assumption, we deem that the result of this test is a **Pass**.